In [2]:
import pandas as pd 
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
from imblearn.over_sampling import SMOTE
from collections import Counter

In [3]:
inputs = torch.load('E:/blackblaze/inputs.dat')

In [6]:
len(inputs)

160993

In [4]:
labels = torch.load('E:/blackblaze/labels.dat')

In [8]:
len(labels)

160993

In [10]:
inputs_som = []
labels_som = []

for i in range(len(inputs)):
    inputs[i] = np.array(inputs[i].reshape(5 * 6))

X_som, y_smo = SMOTE(random_state=42).fit_sample(inputs, labels)

for i in X_som:
    i = torch.Tensor(i)
    inputs_som.append(i.reshape(5, 6))

for i in y_smo:
    if(i == 1):
        labels_som.append(torch.ones(1))
    else:
        labels_som.append(torch.zeros(1))


In [11]:
len(inputs_som)

321110

In [12]:
Counter(y_smo

Counter({1.0: 160555, 0.0: 160555})

In [9]:
import pandas as pd 
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
from imblearn.over_sampling import SMOTE
from collections import Counter

device = torch.device('cuda')
print(device)

class diskLSTM(nn.Module):
    def __init__(self):
        super(diskLSTM, self).__init__()

        self.LSTM_1 = nn.LSTM(input_size=6, hidden_size=32, batch_first=True)
        self.LSTM_2 = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.LSTM_3 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
 
        self.linear_1 = nn.Linear(128, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)
    
    def forward(self, X, state):
        X, _ = self.LSTM_1(X, state)
        X, _ = self.LSTM_2(X, state)
        X, _ = self.LSTM_3(X, state)

        X = X[:, X.size(1) - 1, :]


        X = self.linear_1(X)

        X = torch.tanh(X)
        X = self.linear_2(X)

        X = torch.tanh(X)
        X = self.linear_3(X)

        X = torch.sigmoid(X)

        return X

class Mydata(Dataset):
    def __init__(self, inputs, labels, numstep):

        self.fea_num = 6

        self.X_som = []
        self.y_smo = []

        
        self.inputs_som = []
        self.labels_som = []

  
        for i in range(len(inputs)):
            inputs[i] = np.array(inputs[i].reshape(numstep * self.fea_num))
        
        self.X_som, self.y_smo = SMOTE(random_state=42).fit_sample(inputs, labels)

        for i in self.X_som:
            i = torch.Tensor(i)
            self.inputs_som.append(i.reshape(numstep, self.fea_num))

        for i in self.y_smo:
            if(i == 1):
                self.labels_som.append(torch.ones(1))
            else:
                self.labels_som.append(torch.zeros(1))

    def __len__(self):
        return len(self.inputs_som)

    def __getitem__(self, idx):
        return self.inputs_som[idx], self.labels_som[idx]

    def get(self):
        return self.inputs_som, self.labels_som

def classifyRes(y_hat):
    for i in range(y_hat.size(0)):
        if(y_hat[i] > 0.5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
    return y_hat


def train(net, train_iter, optimizer, num_epochs):
    print("training start")
    loss = torch.nn.BCELoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start, fp, sum_f = 0.0, 0.0, 0, 0, time.time(), 0.0, 0.0
        state = None
        for X, y in train_iter:
            X = X.to(device)
            y = y.squeeze(1)
            y = y.to(device)
            y_hat = net(X.float(), state)
            y_hat = y_hat.to(device)
            l = loss(y_hat, y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            y_hat = classifyRes(y_hat)
            train_acc_sum += (y_hat.cuda() == y).sum().item()
            n += y.size(0)
            batch_count += 1
            for i in range(len(y_hat)):
                if(y[i] == 1 and y[i] == y_hat[i]):
                    fp = fp + 1
            sum_f = sum_f + y.sum().item()
            print('y', y)
            print('y_hat', y_hat.reshape(y_hat.size(0)))
            print('fp: ', fp)
            print('sum_fp: ', sum_f)
            break
        print('epoch %d, loss %.4f, train acc %.3f, time %.1f sec, fp acc %.3f'
        % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, time.time() - start, fp / sum_f))





train_set = Mydata(inputs, labels, 5)
train_iter = DataLoader(train_set, batch_size = 64, shuffle = True)


net = diskLSTM()
net = net.to(device)
lr, num_epochs = 0.1, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

train(net, train_iter, optimizer, num_epochs)





cuda
training start
y tensor([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
        1., 0., 1., 1., 0., 0., 0., 1., 1., 0.], device='cuda:0')
y_hat tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', grad_fn=<ViewBackward>)
fp:  0.0
sum_fp:  33.0
epoch 1, loss 0.6983, train acc 31.000, time 0.1 sec, fp acc 0.000
y tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
     

In [1]:
import numpy as np 

In [3]:
a = np.genfromtxt('E:/blackblaze/positiveSample.csv', dtype = None)

In [4]:
a

array([b&#39;,capacity_bytes,date,failure,model,serial_number,smart_190_raw,smart_193_raw,smart_194_raw,smart_1_raw,smart_241_raw,smart_7_raw&#39;,
       b&#39;0,4.00079E+12,2018/1/1,0,ST4000DM000,W3004X68,17,103203,17,216941544,18981102610,304428807&#39;,
       b&#39;1,4.00079E+12,2018/1/2,0,ST4000DM000,W3004X68,17,103203,17,88620216,18983249064,305190394&#39;,
       ...,
       b&#39;2645,4.00079E+12,2018/9/27,0,ST4000DM000,S300ZRQS,23,6899,23,162741008,48646632335,261367389&#39;,
       b&#39;2646,4.00079E+12,2018/9/28,0,ST4000DM000,S300ZRQS,23,6901,23,159964192,48666406967,263100937&#39;,
       b&#39;2647,4.00079E+12,2018/9/29,1,ST4000DM000,S300ZRQS,23,6901,23,191744272,48676237535,264091278&#39;],
      dtype=&#39;|S128&#39;)

In [12]:
type(a[0])

numpy.bytes_

In [15]:
import torch 

In [16]:
a = torch.tensor([0, 1, 1, 0])

In [17]:
b = torch.tensor([1, 1, 1, 1])

In [18]:
a

tensor([0, 1, 1, 0])

In [19]:
b

tensor([1, 1, 1, 1])

In [20]:
n = 0
for i in range(len(a)):
    if(a[i] == 1 and a[i] == b[i]):
        n = n + 1

In [21]:
n

2

In [31]:
type(a.sum().item())

int

In [15]:
import pandas as pd 
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
from imblearn.over_sampling import SMOTE
from collections import Counter
from torch.autograd import Variable
device = torch.device('cuda')


print(device)

class diskLSTM(nn.Module):
    def __init__(self, batch_size):
        super(diskLSTM, self).__init__()

        self.LSTM_1 = nn.LSTM(input_size=6, hidden_size=32, batch_first=True)
        self.LSTM_2 = nn.LSTM(input_size=32, hidden_size=64, batch_first=True)
        self.LSTM_3 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
 
        self.linear_1 = nn.Linear(128, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)

        self.state_1 = (Variable(torch.randn(1, batch_size, 32).cuda()), Variable(torch.randn(1, batch_size, 32).cuda()))
        self.state_2 = (Variable(torch.randn(1, batch_size, 64).cuda()), Variable(torch.randn(1, batch_size, 64).cuda()))
        self.state_3 = (Variable(torch.randn(1, batch_size, 128).cuda()), Variable(torch.randn(1, batch_size, 128).cuda()))
    
    def forward(self, X):
        X, _ = self.LSTM_1(X, self.state_1)
        X, _ = self.LSTM_2(X, self.state_2)
        X, _ = self.LSTM_3(X, self.state_3)

        X = X[:, X.size(1) - 1, :]


        X = self.linear_1(X)

        X = torch.tanh(X)
        X = self.linear_2(X)

        X = torch.tanh(X)
        X = self.linear_3(X)

        X = torch.sigmoid(X)

        return X

class Mydata(Dataset):
    def __init__(self, inputs, labels, numstep):

        self.fea_num = 6

        self.X_som = []
        self.y_smo = []

        
        self.inputs_som = []
        self.labels_som = []

  
        for i in range(len(inputs)):
            inputs[i] = np.array(inputs[i].reshape(numstep * self.fea_num))
        
        self.X_som, self.y_smo = SMOTE(random_state=42).fit_sample(inputs, labels)

        for i in self.X_som:
            i = torch.Tensor(i)
            self.inputs_som.append(i.reshape(numstep, self.fea_num))

        for i in self.y_smo:
            if(i == 1):
                self.labels_som.append(torch.ones(1))
            else:
                self.labels_som.append(torch.zeros(1))

    def __len__(self):
        return len(self.inputs_som)

    def __getitem__(self, idx):
        return self.inputs_som[idx], self.labels_som[idx]

    def get(self):
        return self.inputs_som, self.labels_som

def classifyRes(y_hat):
    for i in range(y_hat.size(0)):
        if(y_hat[i] > 0.5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
    return y_hat


def train(net, train_iter, optimizer, num_epochs):
    print("training start")
    loss = torch.nn.BCELoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start, sum_fp, fp = 0.0, 0.0, 0, 0, time.time(), 0.0, 0.0
        print('strat sum_fp: ', sum_fp)
        for X, y in train_iter:
            X = X.to(device)
            y = y.squeeze(1)
            y = y.to(device)
            y_hat = net(X.float())
            y_hat = y_hat.reshape(y_hat.size(0))
            print('strat y_hat', y_hat)
            y_hat = y_hat.to(device)
            l = loss(y_hat, y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            y_hat = classifyRes(y_hat)
            train_acc_sum += (y_hat.cuda() == y).sum().item()
            n += y.size(0)
            batch_count += 1
            for i in range(len(y_hat)):
                if(y[i] == 1 and y[i] == y_hat[i]):
                    fp = fp + 1
            sum_fp = sum_fp + y.sum().item()
            print(y.shape)
            print('y ', y)
            print('y_hat', y_hat)
            print(y_hat.shape)
            print('fp: ', fp)
            print('sum_fp: ', sum_fp)
            print('*'*20)
            break
        print('epoch %d, loss %.4f, train acc %.3f, FP acc %.3f, time %.1f sec'
        % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, fp / sum_fp, time.time() - start))
    





train_set = Mydata(inputs, labels, 5)
train_iter = DataLoader(train_set, batch_size = 64, shuffle = True, drop_last=True)


net = diskLSTM(64)
net = net.to(device)
lr, num_epochs = 0.001, 100
optimizer = torch.optim.Adamax(net.parameters(), lr=lr)
print('lr', lr)
train(net, train_iter, optimizer, num_epochs)




cuda


RuntimeError: CUDA error: device-side assert triggered